In [ ]:
# Function to filter out corrupted images
def filter_corrupted_images(folder_path):
    num_skipped = 0
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)
    print(f"Deleted {num_skipped} images in {folder_path}")

In [ ]:
# Replace these with your actual class names
class_names = ["Parkinson", "Healthy"]

In [ ]:
# Specify image size and batch size
image_size = (64, 64)  # Resized image size
batch_size = 128

In [ ]:
# Main folder path
main_folder_path = "G:\\Rifa\\Parkinson\\Created\\Spiral"
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
from tensorflow.keras.layers import SeparableConv2D, MaxPooling2D,
Flatten, Dense, Input, Attention, concatenate
from tensorflow.keras.models import Model

In [ ]:
# Function to extract HOG features from an image
def extract_hog_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fd, _ = hog(gray_image, orientations=8, pixels_per_cell=(8, 8),
cells_per_block=(1, 1), visualize=True)
    return fd.flatten()

In [ ]:
# Main folder path
main_folder_path = "G:\\Rifa\\Parkinson\\Created\\Spiral"

In [ ]:
# Load and preprocess the dataset
images = []
labels = []
class_names = ['Healthy', 'Parkinson']
image_size = (64, 64)

def filter_corrupted_images(folder_path):
    pass  # Add your implementation to filter corrupted images if
needed

for class_name in class_names:
    folder_path = os.path.join(main_folder_path, class_name)
    filter_corrupted_images(folder_path)
    for fname in os.listdir(folder_path):
        img_path = os.path.join(folder_path, fname)
        img = cv2.imread(img_path)
        img = cv2.resize(img, image_size)
        images.append(img)
        labels.append(class_name)

In [ ]:
# Convert labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [ ]:
# Split the dataset into training (70%), validation (15%), and test
(15%)
X_train, X_temp, y_train, y_temp = train_test_split(images, labels,
test_size=0.3, random_state=1337)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp,
test_size=0.5, random_state=1337)

In [ ]:
# Extract HOG features
X_train_hog = np.array([extract_hog_features(img) for img in
X_train])
X_val_hog = np.array([extract_hog_features(img) for img in X_val])
X_test_hog = np.array([extract_hog_features(img) for img in
X_test])

In [ ]:
# Define input shape for the CNN
image_size = (64, 64)

In [ ]:
# Define the input layer for the CNN
cnn_input = Input(shape=(image_size[0], image_size[1], 3))

In [ ]:
# Define CNN layers
conv1 = SeparableConv2D(32, (3, 3), activation='gelu')(cnn_input)
max_pool1 = MaxPooling2D((2, 2))(conv1)

In [ ]:
# Add Attention layer
attention1 = Attention()([max_pool1, max_pool1])

In [ ]:
# Define the rest of the CNN layers
conv2 = SeparableConv2D(64, (3, 3), activation='gelu')(attention1)
max_pool2 = MaxPooling2D((2, 2))(conv2)
flatten = Flatten()(max_pool2)
dense = Dense(64, activation='gelu')(flatten)

In [ ]:
# Create the CNN model
cnn_model = Model(inputs=cnn_input, outputs=dense)

In [ ]:
# Build a fully connected model for HOG features
hog_input = Input(shape=(X_train_hog.shape[1],))
hog_model = Dense(64, activation='gelu')(hog_input)

In [ ]:
# Concatenate the CNN and HOG models
concatenated = concatenate([cnn_model.output, hog_model])

In [ ]:
# Add more layers as needed
merged_model = Dense(64, activation='gelu')(concatenated)
output = Dense(len(class_names),
activation='sigmoid')(merged_model)

In [ ]:
# Create the final model
final_model = Model(inputs=[cnn_model.input, hog_input],
outputs=output)

In [ ]:
# Example usage:
final_model.summary()

In [ ]:
# Compile the model
final_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rat
 e=0.0001),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

In [ ]:
# Convert image lists to numpy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

In [ ]:
# Rescale and normalize images
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

In [ ]:
# Train the model for 50 epochs
history = final_model.fit([X_train, X_train_hog], y_train,
                           validation_data=([X_val, X_val_hog],y_val),
                           epochs=50)

In [ ]:
# Visualize the training history (accuracy and loss)
plt.figure(figsize=(12, 4))

In [ ]:
# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation
Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = final_model.evaluate([X_test,
X_test_hog], y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# Predictions
test_preds = final_model.predict([X_test, X_test_hog])
test_labels = []

for label in y_test:
    test_labels.append(label)  # Append the label directly

In [ ]:
# Confusion matrix
conf_mat = confusion_matrix(test_labels, np.argmax(test_preds,
axis=1))
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='g', cmap='Blues',
xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()

In [ ]:
# Classification report
print("Classification Report:")
print(classification_report(test_labels, np.argmax(test_preds,
axis=1), target_names=class_names))

In [ ]:
# ROC Curve
fpr, tpr, _ = roc_curve(test_labels, test_preds[:, 1])
roc_auc = auc(fpr, tpr)

In [ ]:
# Plot ROC Curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area
= {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Save the model as .h5 file
final_model.save('parkinson_model.h5')

In [ ]:
# Load the model from .h5 file
loaded_model = models.load_model('parkinson_model.h5')